<a href="https://colab.research.google.com/github/ghhyc/Project-4/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The goal was to first Extract the two types of datasets Heart Disease and Multi Disease. Each dataset contained multiple .csv files which had to be Transformed by renaming and joining columns. Once Extraction and Transformation was complete,Loading part was accomplished.

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-26 23:30:18--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.23MB/s    in 0.7s    

2022-02-26 23:30:20 (1.23 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Heart_Disease_predictions").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

#Load Heart Disease data and combine to one dataframe.

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://data-bootcamp-01.s3.us-west-1.amazonaws.com/Heart_Disease_Prediction.csv"
spark.sparkContext.addFile(url)
prediction_df = spark.read.csv(SparkFiles.get("Heart_Disease_Prediction.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
prediction_df.show()

+---+---+---------------+---+-----------+------------+-----------+------+---------------+-------------+-----------+--------------------+--------+-------------+
|age|Sex|Chest_pain_type| BP|Cholesterol|FBS_over_120|EKG_results|Max_HR|Exercise_angina|ST_depression|Slope_of_ST|Number_vessels_fluro|Thallium|Heart_Disease|
+---+---+---------------+---+-----------+------------+-----------+------+---------------+-------------+-----------+--------------------+--------+-------------+
| 70|  1|              4|130|        322|           0|          2|   109|              0|          2.4|          2|                   3|       3|     Presence|
| 67|  0|              3|115|        564|           0|          2|   160|              0|          1.6|          2|                   0|       7|      Absence|
| 57|  1|              2|124|        261|           0|          0|   141|              0|          0.3|          1|                   0|       7|     Presence|
| 64|  1|              4|128|        263

In [5]:
url="https://data-bootcamp-01.s3.us-west-1.amazonaws.com/heart_failure_clinical_records_dataset.csv"
spark.sparkContext.addFile(url)
clinical_df = spark.read.csv(SparkFiles.get("heart_failure_clinical_records_dataset.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
clinical_df.show()

+----+-------+------------------------+--------+-----------------+-------------------+---------+----------------+------------+---+-------+----+-----------+
| age|anaemia|creatinine_phosphokinase|diabetes|ejection_fraction|high_blood_pressure|platelets|serum_creatinine|serum_sodium|sex|smoking|time|DEATH_EVENT|
+----+-------+------------------------+--------+-----------------+-------------------+---------+----------------+------------+---+-------+----+-----------+
|75.0|      0|                     582|       0|               20|                  1| 265000.0|             1.9|         130|  1|      0|   4|          1|
|55.0|      0|                    7861|       0|               38|                  0|263358.03|             1.1|         136|  1|      0|   6|          1|
|65.0|      0|                     146|       0|               20|                  0| 162000.0|             1.3|         129|  1|      1|   7|          1|
|50.0|      1|                     111|       0|               2

In [6]:
url="https://data-bootcamp-01.s3.us-west-1.amazonaws.com/heart.csv"
spark.sparkContext.addFile(url)
heart_df = spark.read.csv(SparkFiles.get("heart.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
heart_df.show()

+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
|age|sex| cp|trtbps|chol|fbs|restecg|thalachh|exng|oldpeak|slp|caa|thall|output|
+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
| 63|  1|  3|   145| 233|  1|      0|     150|   0|    2.3|  0|  0|    1|     1|
| 37|  1|  2|   130| 250|  0|      1|     187|   0|    3.5|  0|  0|    2|     1|
| 41|  0|  1|   130| 204|  0|      0|     172|   0|    1.4|  2|  0|    2|     1|
| 56|  1|  1|   120| 236|  0|      1|     178|   0|    0.8|  2|  0|    2|     1|
| 57|  0|  0|   120| 354|  0|      1|     163|   1|    0.6|  2|  0|    2|     1|
| 57|  1|  0|   140| 192|  0|      1|     148|   0|    0.4|  1|  0|    1|     1|
| 56|  0|  1|   140| 294|  0|      0|     153|   0|    1.3|  1|  0|    2|     1|
| 44|  1|  1|   120| 263|  0|      1|     173|   0|    0.0|  2|  0|    3|     1|
| 52|  1|  2|   172| 199|  1|      1|     162|   0|    0.5|  2|  0|    3|     1|
| 57|  1|  2|   150| 168|  0

#Join the prediction_df and clinical_df via "age"

In [8]:
# Join the  DataFrames
joined_df= prediction_df.join(clinical_df, on="age", how="inner")
joined_df.show()

+---+---+---------------+---+-----------+------------+-----------+------+---------------+-------------+-----------+--------------------+--------+-------------+-------+------------------------+--------+-----------------+-------------------+---------+----------------+------------+---+-------+----+-----------+
|age|Sex|Chest_pain_type| BP|Cholesterol|FBS_over_120|EKG_results|Max_HR|Exercise_angina|ST_depression|Slope_of_ST|Number_vessels_fluro|Thallium|Heart_Disease|anaemia|creatinine_phosphokinase|diabetes|ejection_fraction|high_blood_pressure|platelets|serum_creatinine|serum_sodium|sex|smoking|time|DEATH_EVENT|
+---+---+---------------+---+-----------+------------+-----------+------+---------------+-------------+-----------+--------------------+--------+-------------+-------+------------------------+--------+-----------------+-------------------+---------+----------------+------------+---+-------+----+-----------+
| 55|  1|              4|132|        353|           0|          0|   132|

#Join joined_df with heart_df via "age"

In [12]:
# Join the  DataFrames
heart_joined_df= joined_df.join(heart_df, on="age", how="inner")
heart_joined_df.show()

+---+---+---------------+---+-----------+------------+-----------+------+---------------+-------------+-----------+--------------------+--------+-------------+-------+------------------------+--------+-----------------+-------------------+---------+----------------+------------+---+-------+----+-----------+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
|age|Sex|Chest_pain_type| BP|Cholesterol|FBS_over_120|EKG_results|Max_HR|Exercise_angina|ST_depression|Slope_of_ST|Number_vessels_fluro|Thallium|Heart_Disease|anaemia|creatinine_phosphokinase|diabetes|ejection_fraction|high_blood_pressure|platelets|serum_creatinine|serum_sodium|sex|smoking|time|DEATH_EVENT|sex| cp|trtbps|chol|fbs|restecg|thalachh|exng|oldpeak|slp|caa|thall|output|
+---+---+---------------+---+-----------+------------+-----------+------+---------------+-------------+-----------+--------------------+--------+-------------+-------+------------------------+--------+-----------------+-------------

In [13]:
print(heart_joined_df.count())

19177


In [14]:
heart_joined_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- Sex: integer (nullable = true)
 |-- Chest_pain_type: integer (nullable = true)
 |-- BP: integer (nullable = true)
 |-- Cholesterol: integer (nullable = true)
 |-- FBS_over_120: integer (nullable = true)
 |-- EKG_results: integer (nullable = true)
 |-- Max_HR: integer (nullable = true)
 |-- Exercise_angina: integer (nullable = true)
 |-- ST_depression: double (nullable = true)
 |-- Slope_of_ST: integer (nullable = true)
 |-- Number_vessels_fluro: integer (nullable = true)
 |-- Thallium: integer (nullable = true)
 |-- Heart_Disease: string (nullable = true)
 |-- anaemia: integer (nullable = true)
 |-- creatinine_phosphokinase: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- ejection_fraction: integer (nullable = true)
 |-- high_blood_pressure: integer (nullable = true)
 |-- platelets: double (nullable = true)
 |-- serum_creatinine: double (nullable = true)
 |-- serum_sodium: integer (nullable = true)
 |-- sex: integer 

#Load Disease Data

In [17]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://data-bootcamp-01.s3.us-west-1.amazonaws.com/disease_occurrences.csv"
spark.sparkContext.addFile(url)
occurrences_df = spark.read.csv(SparkFiles.get("disease_occurrences.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
occurrences_df.show()

+---+--------------------+----+
|_c0|                   0|   1|
+---+--------------------+----+
|  0|hypertensive  dis...|3363|
|  1|            diabetes|1421|
|  2|  depression  mental|1337|
|  3| depressive disorder|1337|
|  4|coronary  arterio...|1284|
|  5|coronary heart di...|1284|
|  6|           pneumonia|1029|
|  7|failure  heart co...| 963|
|  8|accident  cerebro...| 885|
|  9|              asthma| 835|
| 10|myocardial  infar...| 759|
| 11|hypercholesterolemia| 685|
| 12|           infection| 630|
| 13|infection  urinar...| 597|
| 14|              anemia| 544|
| 15|chronic  obstruct...| 524|
| 16|            dementia| 504|
| 17|insufficiency  renal| 445|
| 18|           confusion| 408|
| 19|degenerative  pol...| 405|
+---+--------------------+----+
only showing top 20 rows



#Renamed the above columns to "count", "diseases", "total"

In [23]:
occurrences_df=occurrences_df.withColumnRenamed("_c0", "count")\
                             .withColumnRenamed("0", "diseases")\
                             .withColumnRenamed("1", "total")
occurrences_df.show(10)

+-----+--------------------+-----+
|count|            diseases|total|
+-----+--------------------+-----+
|    0|hypertensive  dis...| 3363|
|    1|            diabetes| 1421|
|    2|  depression  mental| 1337|
|    3| depressive disorder| 1337|
|    4|coronary  arterio...| 1284|
|    5|coronary heart di...| 1284|
|    6|           pneumonia| 1029|
|    7|failure  heart co...|  963|
|    8|accident  cerebro...|  885|
|    9|              asthma|  835|
+-----+--------------------+-----+
only showing top 10 rows



In [18]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://data-bootcamp-01.s3.us-west-1.amazonaws.com/disease_symptoms.csv"
spark.sparkContext.addFile(url)
symptoms_df = spark.read.csv(SparkFiles.get("disease_symptoms.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
symptoms_df.show()

+---+--------------------+--------------------+
|_c0|                   0|                   1|
+---+--------------------+--------------------+
|  0|hypertensive  dis...|['pain  chest', '...|
|  1|            diabetes|['polyuria', 'pol...|
|  2|  depression  mental|['feeling  suicid...|
|  3| depressive disorder|['feeling  suicid...|
|  4|coronary  arterio...|['pain  chest', '...|
|  5|coronary heart di...|['pain  chest', '...|
|  6|           pneumonia|['cough', 'fever'...|
|  7|failure  heart co...|['shortness  of b...|
|  8|accident  cerebro...|['dysarthria', 'a...|
|  9|              asthma|['wheezing', 'cou...|
| 10|myocardial  infar...|['pain chest', 's...|
| 11|hypercholesterolemia|['pain', 'pain ch...|
| 12|           infection|['fever', 'erythe...|
| 13|infection  urinar...|['fever', 'dysuri...|
| 14|              anemia|['chill', 'guaiac...|
| 15|chronic  obstruct...|['shortness of br...|
| 16|            dementia|['fever', 'fall',...|
| 17|insufficiency  renal|['shortness of

#Renamed the above columns to "count", "diseases", "total"

In [24]:
symptoms_df=symptoms_df.withColumnRenamed("_c0", "count")\
                             .withColumnRenamed("0", "diseases")\
                             .withColumnRenamed("1", "symptoms")
symptoms_df.show(10)

+-----+--------------------+--------------------+
|count|            diseases|            symptoms|
+-----+--------------------+--------------------+
|    0|hypertensive  dis...|['pain  chest', '...|
|    1|            diabetes|['polyuria', 'pol...|
|    2|  depression  mental|['feeling  suicid...|
|    3| depressive disorder|['feeling  suicid...|
|    4|coronary  arterio...|['pain  chest', '...|
|    5|coronary heart di...|['pain  chest', '...|
|    6|           pneumonia|['cough', 'fever'...|
|    7|failure  heart co...|['shortness  of b...|
|    8|accident  cerebro...|['dysarthria', 'a...|
|    9|              asthma|['wheezing', 'cou...|
+-----+--------------------+--------------------+
only showing top 10 rows



In [20]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://data-bootcamp-01.s3.us-west-1.amazonaws.com/disease_symptoms_network.csv"
spark.sparkContext.addFile(url)
symptoms_network_df = spark.read.csv(SparkFiles.get("disease_symptoms_network.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
symptoms_network_df.show()

+---+--------------------+--------------------+
|_c0|            diseases|            symptoms|
+---+--------------------+--------------------+
|  0|hypertensive  dis...|         pain  chest|
|  1|hypertensive  dis...|shortness  of breath|
|  2|hypertensive  dis...|           dizziness|
|  3|hypertensive  dis...|            asthenia|
|  4|hypertensive  dis...|                fall|
|  5|hypertensive  dis...|             syncope|
|  6|hypertensive  dis...|             vertigo|
|  7|hypertensive  dis...|sweat sweating  i...|
|  8|hypertensive  dis...|         palpitation|
|  9|hypertensive  dis...|              nausea|
| 10|hypertensive  dis...|    angina  pectoris|
| 11|hypertensive  dis...|     pressure  chest|
| 12|            diabetes|            polyuria|
| 13|            diabetes|          polydypsia|
| 14|            diabetes|shortness  of breath|
| 15|            diabetes|         pain  chest|
| 16|            diabetes|            asthenia|
| 17|            diabetes|              

In [25]:
symptoms_network_df=symptoms_df.withColumnRenamed("_c0", "count")
                             
symptoms_network_df.show(10)

+-----+--------------------+--------------------+
|count|            diseases|            symptoms|
+-----+--------------------+--------------------+
|    0|hypertensive  dis...|['pain  chest', '...|
|    1|            diabetes|['polyuria', 'pol...|
|    2|  depression  mental|['feeling  suicid...|
|    3| depressive disorder|['feeling  suicid...|
|    4|coronary  arterio...|['pain  chest', '...|
|    5|coronary heart di...|['pain  chest', '...|
|    6|           pneumonia|['cough', 'fever'...|
|    7|failure  heart co...|['shortness  of b...|
|    8|accident  cerebro...|['dysarthria', 'a...|
|    9|              asthma|['wheezing', 'cou...|
+-----+--------------------+--------------------+
only showing top 10 rows



#Join occurrences_df, symptoms_df via count.

In [34]:
# Join the  DataFrames
s1_joined_df= occurrences_df.join(symptoms_df, on="count", how="inner")

s1_joined_df.show()

+-----+--------------------+-----+--------------------+--------------------+
|count|            diseases|total|            diseases|            symptoms|
+-----+--------------------+-----+--------------------+--------------------+
|    0|hypertensive  dis...| 3363|hypertensive  dis...|['pain  chest', '...|
|    1|            diabetes| 1421|            diabetes|['polyuria', 'pol...|
|    2|  depression  mental| 1337|  depression  mental|['feeling  suicid...|
|    3| depressive disorder| 1337| depressive disorder|['feeling  suicid...|
|    4|coronary  arterio...| 1284|coronary  arterio...|['pain  chest', '...|
|    5|coronary heart di...| 1284|coronary heart di...|['pain  chest', '...|
|    6|           pneumonia| 1029|           pneumonia|['cough', 'fever'...|
|    7|failure  heart co...|  963|failure  heart co...|['shortness  of b...|
|    8|accident  cerebro...|  885|accident  cerebro...|['dysarthria', 'a...|
|    9|              asthma|  835|              asthma|['wheezing', 'cou...|

#Join s1 and symptoms_network_df via count.

In [36]:
# Join the  DataFrames
s2_joined_df= s1_joined_df.join(symptoms_network_df, on="count", how="inner")

s2_joined_df.show()

+-----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|count|            diseases|total|            diseases|            symptoms|            diseases|            symptoms|
+-----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|    0|hypertensive  dis...| 3363|hypertensive  dis...|['pain  chest', '...|hypertensive  dis...|['pain  chest', '...|
|    1|            diabetes| 1421|            diabetes|['polyuria', 'pol...|            diabetes|['polyuria', 'pol...|
|    2|  depression  mental| 1337|  depression  mental|['feeling  suicid...|  depression  mental|['feeling  suicid...|
|    3| depressive disorder| 1337| depressive disorder|['feeling  suicid...| depressive disorder|['feeling  suicid...|
|    4|coronary  arterio...| 1284|coronary  arterio...|['pain  chest', '...|coronary  arterio...|['pain  chest', '...|
|    5|coronary heart di...| 1284|coronary heart

In [37]:
s2_joined_df.printSchema()

root
 |-- count: integer (nullable = true)
 |-- diseases: string (nullable = true)
 |-- total: integer (nullable = true)
 |-- diseases: string (nullable = true)
 |-- symptoms: string (nullable = true)
 |-- diseases: string (nullable = true)
 |-- symptoms: string (nullable = true)



In [19]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://data-bootcamp-01.s3.us-west-1.amazonaws.com/final_disease_symptom_data.csv"
spark.sparkContext.addFile(url)
final_symptoms_df = spark.read.csv(SparkFiles.get("final_disease_symptom_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
final_symptoms_df.show()

+---+--------------------+---------------+-------------+-------------+-------------+------------------+--------------------+------------------+----------------------------+---------------------------+--------+-----------------+----------------+---------+----+--------------+----------------+---------+---------------+---------------------+-----------------------------+-------------+----------------+---------------+--------+-------+-------+---------+----------+-------+---------+--------+------------+------+------+----+---------------+-------------+---------+--------------------+-----------------------------------------+--------+------+------------------+----------------------+-----------+------------+-----------------+------------------------+-----------------------+--------------------+-------------------+-----+------------------+-----------------+--------+------------+-----------------------+--------------------+---------+---------------+------------+----------------+----------------+--

In [38]:
final_symptoms_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- diseases: string (nullable = true)
 |-- Heberden's node: integer (nullable = true)
 |-- Murphy's sign: integer (nullable = true)
 |-- Stahli's line: integer (nullable = true)
 |-- abdomen acute: integer (nullable = true)
 |-- abdominal bloating: integer (nullable = true)
 |-- abdominal tenderness: integer (nullable = true)
 |-- abnormal sensation: integer (nullable = true)
 |-- abnormally  hard consistency: integer (nullable = true)
 |-- abnormally hard consistency: integer (nullable = true)
 |-- abortion: integer (nullable = true)
 |-- abscess bacterial: integer (nullable = true)
 |-- absences finding: integer (nullable = true)
 |-- achalasia: integer (nullable = true)
 |-- ache: integer (nullable = true)
 |-- adverse effect: integer (nullable = true)
 |-- adverse reaction: integer (nullable = true)
 |-- agitation: integer (nullable = true)
 |-- air fluid level: integer (nullable = true)
 |-- alcohol binge episode: integer (nullable = true

#Next steps:

1. Team will check if ETL process was correctly accomplished.

2. Discuss the two dataset and comeup with thesis to apply ML techniques.